# Service Graph Analysis

Build and analyze the microservice dependency graph.

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from graph_builder import ServiceGraphBuilder

%matplotlib inline

## 1. Load Service Pairs

In [ ]:
# Load preprocessed service pairs
pairs = pd.read_csv('../data/processed/service_pairs.csv')
print(f"Loaded {len(pairs)} service pairs")
pairs.head()

## 2. Build Dependency Graph

In [ ]:
# Build graph
builder = ServiceGraphBuilder()
graph = builder.build_from_pairs(pairs)

print(f"\nGraph Statistics:")
print(f"Nodes (services): {graph.number_of_nodes()}")
print(f"Edges (dependencies): {graph.number_of_edges()}")
print(f"Density: {nx.density(graph):.4f}")

## 3. Compute Graph Metrics

In [ ]:
# Compute metrics
metrics = builder.compute_metrics()

print("Graph Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

## 4. Analyze Service Centrality

In [ ]:
# Degree centrality
in_degree = dict(graph.in_degree())
out_degree = dict(graph.out_degree())

# Sort by in-degree (most called services)
top_called = sorted(in_degree.items(), key=lambda x: x[1], reverse=True)[:10]

print("Top 10 Most Called Services:")
for service, degree in top_called:
    print(f"  {service}: {degree} incoming calls")

# PageRank
pagerank = nx.pagerank(graph)
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 Services by PageRank:")
for service, score in top_pagerank:
    print(f"  {service}: {score:.4f}")

## 5. Identify Bottlenecks

In [ ]:
# Find bottlenecks
bottlenecks = builder.identify_bottlenecks(latency_threshold=100)

print(f"Found {len(bottlenecks)} potential bottlenecks")
print("\nTop 5 Bottlenecks:")
for caller, callee, latency in bottlenecks[:5]:
    print(f"  {caller} -> {callee}: {latency:.2f}ms")

## 6. Visualize Graph

In [ ]:
# Visualize the dependency graph
builder.visualize(layout='spring', figsize=(20, 15))

## 7. Detect Circular Dependencies

In [ ]:
# Find cycles
try:
    cycles = list(nx.simple_cycles(graph))
    print(f"Found {len(cycles)} circular dependencies")
    
    if cycles:
        print("\nCircular Dependencies:")
        for i, cycle in enumerate(cycles[:5]):
            print(f"  {i+1}. {' -> '.join(cycle)} -> {cycle[0]}")
except:
    print("No cycles found")